# Import Libraries and Load Environment Variables

In [3]:
import json
import os
from dotenv import load_dotenv
import nest_asyncio
from llama_parse import LlamaParse

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain.schema import Document
from langchain_google_genai import ChatGoogleGenerativeAI
import tiktoken

nest_asyncio.apply()
load_dotenv(override=True)

data_dir = "input_big_context/"

llm = ChatGoogleGenerativeAI( model="gemini-1.5-flash", temperature=0, max_retries=2)

# Parse PDF Documents to Markdown

In [5]:

for file in os.listdir(data_dir):
    if file.endswith(".pdf"):
        try:
            print(f"Converting {file} to markdown")
            md_text = LlamaParse(
                result_type="markdown", 
                verbose=True,
                #use_vendor_multimodal_model=True,
                #vendor_multimodal_model_name="openai-gpt-4o-mini",
                #vendor_multimodal_api_key=os.getenv("OPENAI_API_KEY"),
                language="en",
                numWorkers=5).load_data(data_dir + file)
            combined_md_text = "\n\n".join([doc.text for doc in md_text])
            md_file_path = data_dir + file.replace(".pdf", ".md")
            print(f"Saving markdown to {md_file_path}")
            with open(md_file_path, "w", encoding="utf-8") as f:
                f.write(combined_md_text)
            print(f"Successfully converted {file}")
        except Exception as e:
            print(f"Error converting {file}: {e}")

Converting bcg-accelerating-climate-action-with-ai-nov-2023-rev.pdf to markdown
Started parsing the file under job_id b236c775-efff-4ce1-8a68-d36df9f798da
..Saving markdown to input_big_context/bcg-accelerating-climate-action-with-ai-nov-2023-rev.md
Successfully converted bcg-accelerating-climate-action-with-ai-nov-2023-rev.pdf
Converting SAP Integrated Report 2023.pdf to markdown
Started parsing the file under job_id fc2a1afb-fdba-489a-b5eb-ce9893c1fe7a
...Saving markdown to input_big_context/SAP Integrated Report 2023.md
Successfully converted SAP Integrated Report 2023.pdf
Converting the-economic-potential-of-generative-ai-the-next-productivity-frontier.pdf to markdown
Started parsing the file under job_id 4b59acbb-a5d1-4f32-b40b-70ff8736ba60
..Saving markdown to input_big_context/the-economic-potential-of-generative-ai-the-next-productivity-frontier.md
Successfully converted the-economic-potential-of-generative-ai-the-next-productivity-frontier.pdf
Converting the-state-of-ai-in-ear

In [ ]:
encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(combined_md_text)
print(len(tokens))

with open(md_file_path, "w", encoding="utf-8") as f:
    f.write(combined_md_text)
    print(f"Successfully converted {file}")

Successfully converted s10668-023-02933-7.pdf


# Generate Dataset from Data

In [22]:
from typing import List
from pydantic import BaseModel, Field
import tiktoken

class Response(BaseModel):
    question: str = Field(description="Question")
    ground_truth: str = Field(description="Ground Truth")
    references: List[str] = Field(description="List of document or reference names used")

class Responses(BaseModel):
    responses: list[Response] = Field(description="List of responses")

documents = []
for file in os.listdir(data_dir):
    if file.endswith(".md"):
        file_path = os.path.join(data_dir, file)
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
            documents.append(Document(page_content=content, metadata={"source": file}))
combined_documents_content = "\n\n".join([doc.page_content for doc in documents])

parser = PydanticOutputParser(pydantic_object=Responses)

prompt = PromptTemplate(
    input_variables=["documents"],
    partial_variables={"format_output": parser.get_format_instructions()},
    template="""
You are tasked with generating 100 realistic and diverse business questions based on the documents provided below, which include SAP internal documents and other reports about varios business topics. 
Each question should be written from the perspective of SAP, focusing on what SAP can do to achieve specific business objectives (e.g., increase margins, improve sustainability, enhance product offerings), considering various factors.

Integration of Diverse and Unlinked Knowledge Areas:
- Each question must blend insights from multiple business domains (e.g., finance, sustainability, technology, operations)
- Include concepts that are not necessarily semantically linked

Multi-hop Information Synthesis Within and Across Documents:
- Answers should require synthesizing multiple pieces of relevant information.
- Draw information from multiple documents and multiple sections within a single document.
- Ensure that multi-step reasoning is necessary to combine various information fragments into a comprehensive answer.

Abstract Reasoning and Insight Generation:
- Questions should push beyond simple fact retrieval and basic sensemaking.
- Require abstract reasoning
- Question should be global themed and require a understand of the whole documents to be answered

For Each Question, Provide the Following:

Question: 
- Formulate a complex business question that meets all the above constraints.

Optimal Answer (Ground Truth):
- Provide a comprehensive answer to the question.
- The answer should synthesize all relevent information from the relevant documents and include abstract reasoning.

Source Referenced:
- Provide an array/list of the document names or identifiers that were used to form the answer.

Question Examples:
1. How can we capitalize on eco-friendly manufacturing practices to both improve our sustainability credentials and boost our financial performance?
2. What innovative products should we consider integrating into our portfolio, and which current products have the highest potential for enhancement through machine learning technologies?

The output MUST strictly adhere to the following JSON format, and NO other text MUST be included:    
{format_output}

Use only the following documents: 
<documents>
{documents}
</documents>
"""
)

chain = prompt | llm | parser

encoding = tiktoken.encoding_for_model("gpt-4o")

tokens = encoding.encode(combined_documents_content)

print(len(tokens))

186499


In [23]:
response = chain.invoke({"documents": combined_documents_content})

Save LLM response to file

In [ ]:
with open("./synthetic_data_big_context_test_citet2.json", "w", encoding="utf-8") as f:
    json.dump(response.dict(), f, indent=4)

: 